# **Movie Production Business Analysis - IMDB ETL**

- Yvon Bilodeau
- May 2022

## **Business Problem**

For this project, you have been hired to produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. Ultimately, you will use this database to analyze what makes a movie successful and will provide recommendations to the stakeholder on how to make a successful movie.


## **The Data**

IMDB Provides Several Files with varied information for Movies, TV Shows, Made for TV Movies, etc.

Overview/Data Dictionary: [https://www.imdb.com/interfaces/](https://www.imdb.com/interfaces/)

Downloads page: [https://datasets.imdbws.com/](https://datasets.imdbws.com/)
        
From their previous research, they realized they want to focus on the following files:

- title.basics.tsv.gz
- title.ratings.tsv.gz
- title.akas.tsv.gz

### **Specifications**

Your stakeholder only wants you to include information for movies based on the following specifications:

- Exclude any movie with missing values for genre or runtime
- Include only full-length movies (titleType = "movie").
- Include only fictional movies (not from documentary genre)
- Include only movies that were released 2000 - 2021 (include 2000 and 2021)
- Include only movies that were released in the United States

### **Download**

In [1]:
# Create url variables
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
ratings_url ="https://datasets.imdbws.com/title.ratings.tsv.gz"
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
#akas_url = "C:\Users\DELL\Downloads\title.akas.tsv.gz"

### **Preprocessing**

In [2]:
import numpy as np
import pandas as pd
import os

#### **Title Basics**

In [ ]:
# Create dataframe
basics_df = pd.read_csv(basics_url,sep='\t', low_memory=False)

- Replace "\N" with np.nan
- Eliminate movies that are null for runtimeMinutes
- Eliminate movies that are null for genre
- Keep only titleType==Movie
- Keep startYear 2000-2022
- Eliminate movies that include  "Documentary" in genre

In [ ]:
basics_df.head()

In [ ]:
# Replace "\N" with np.nan
basics_df.replace({'\\N':np.nan},inplace=True)

In [ ]:
basics_df.head()

In [ ]:
# Eliminate movies that are null for runtimeMinutes
basics_df = basics_df.dropna(subset = ['runtimeMinutes', 'genres', 'startYear'])

In [ ]:
# Keep only titleType == Movie
basics_df = basics_df.loc[basics_df['titleType'] == 'movie']

In [ ]:
basics_df.head(20)

In [ ]:
# Keep startYear 2000-2022
basics_df['startYear'] = basics_df['startYear'].astype(int)
basics_df = basics_df.loc[(basics_df['startYear'] >= 2000) & (basics_df['startYear'] <=2021)]

In [ ]:
basics_df.head()

In [ ]:
# Eliminate movies that include "Documentary" in genre
documentary_filter = basics_df['genres'].str.contains('documentary', case=False)
basics_df = basics_df[~documentary_filter]

In [ ]:
basics_df.head(20)

In [ ]:
# Make directory
os.makedirs('Data/',exist_ok=True) 

In [ ]:
# Confirm directory created
os.listdir("Data/")

In [ ]:
## Save dataframe to file
basics_df.to_csv("Data/title_basics.csv.gz", compression='gzip', index=False)

In [ ]:
# Open saved file and preview again
basics_df = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics_df.head()

In [ ]:
del basics_df

#### **Title Ratings**

- Keep only US entries.
- Replace "\N" with np.nan

In [ ]:
# Create dataframe
ratings_df = pd.read_csv(ratings_url,sep='\t', low_memory=False)

In [ ]:
ratings_df.head()

In [ ]:
# Replace "\N" with np.nan
ratings_df.replace({'\\N':np.nan},inplace=True)

In [ ]:
ratings_df.info()

In [ ]:
## Save dataframe to file
ratings_df.to_csv("Data/title_ratings.csv.gz", compression='gzip', index=False)

In [ ]:
# Open saved file and preview again
ratings_df = pd.read_csv("Data/title_ratings.csv.gz", low_memory=False)
ratings_df.head()

In [ ]:
del ratings_df

#### **Title Akas**

- Replace "\N" with np.nan (if any)

In [3]:
# Create dataframe # chunksize creates an iterator
dl_file = "C:/Users/DELL/Downloads/title.akas.tsv.gz"
#akas_df = pd.read_csv(dl_file,sep='\t', low_memory=True, chunksize=100000)
akas_df = pd.read_csv(dl_file,sep='\t', low_memory=True)

C:\Users\DELL\anaconda3\envs\dojo-env\lib\site-packages\IPython\core\interactiveshell.py:3524: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
akas_df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [5]:
# Replace "\N" with np.nan
akas_df.replace({'\\N':np.nan},inplace=True)

In [6]:
akas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31775238 entries, 0 to 31775237
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 1.9+ GB


In [7]:
## Save dataframe to file
akas_df.to_csv("Data/title_akas.csv.gz", compression='gzip', index=False)

In [9]:
# Open saved file and preview again
akas_df = pd.read_csv("Data/title_akas.csv.gz", low_memory=True)
akas_df.head()

C:\Users\DELL\anaconda3\envs\dojo-env\lib\site-packages\IPython\core\interactiveshell.py:3524: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0.0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0.0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0.0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0.0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0.0


In [ ]:
# # 31,775,237
# for df in akas_df:
#     display(df.head(), df.tail())

In [ ]:
akas_df.f

In [ ]:
df = akas_df.get_chunk()
df

In [ ]:
# Replace "\N" with np.nan
akas_df.replace({'\\N':np.nan},inplace=True)

In [ ]:
df.to_csv("data/test_akas.csv.gz", compression="gzip", index=False, mode="a")

In [ ]:
df=akas_df.get_chunk()
df.to_csv("data/test_akas.csv.gz", compression="gzip", index=False, mode="a")

In [ ]:
df=pd.read_csv("data/test_akas.csv.gz")
df

In [ ]:
# Save dataframe to file
akas_df.to_csv("Data/title_akas.csv.gz", compression='gzip', index=False)

In [ ]:
# Open saved file and preview again
akas_df = pd.read_csv("Data/title_akas.csv.gz", low_memory=False)
akas_df.head()

In [ ]:
# Open saved file and preview again
# basics_df = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
# basics_df.head()

In [ ]:
# Filter the basics table down to only include the US by using the filter akas dataframe
#keepers = basics_df['tconst'].isin(akas_df['titleId'])
#keepers

In [ ]:
#basics_df = basics_df[keepers]
#basics_df